In [ ]:
import json

with open('steam_games.json', 'r', encoding='utf-8', errors='replace') as f:
    data = json.load(f)

with open('steam_games-2.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4)
# print(data)


In [12]:
from pymongo import MongoClient
import gridfs
import json

# Подключение к MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client.steam  # Используем базу данных 'steam'

# Создаем экземпляр GridFS
fs = gridfs.GridFS(db)

# Путь к вашему большому файлу
large_file_path = 'steam_games-2.json'  # Укажите путь к вашему файлу steam_games.json

# Загрузка файла в GridFS
with open(large_file_path, 'rb') as file:
    file_id = fs.put(file, filename='steam_games-2.json')
    print(f'Large file uploaded with ID: {file_id}')

# Извлечение файла и загрузка данных в коллекцию
file_data = fs.get(file_id)

# Чтение содержимого JSON файла
data = json.load(file_data)

# Вставка данных в коллекцию 'games'
if isinstance(data, list):  # Проверяем, является ли data списком
    db.games.insert_many(data)  # Вставляем все игры
    print(f'Inserted {len(data)} games into the database.')
else:
    print('The JSON file does not contain a list of games.')

# # Удаление файла из GridFS (если это необходимо)
# fs.delete(file_id)
# print('Large file deleted from GridFS.')


Large file uploaded with ID: 67364af111c199b79237af04
The JSON file does not contain a list of games.


In [7]:
from pymongo import MongoClient
import gridfs
import json

ImportError: cannot import name 'MongoClient' from 'pymongo' (c:\Users\nephelim\Desktop\skill\DataCollection\lesson3\pymongo.py)

In [13]:
from pymongo import MongoClient
import gridfs
import json

# Подключение к MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client.steam  # Используем базу данных 'steam'

# Создаем экземпляр GridFS
fs = gridfs.GridFS(db)

# Путь к вашему файлу steam_games.json
large_file_path = 'steam_games.json'  # Укажите путь к вашему файлу steam_games.json

# Загрузка файла в GridFS
with open(large_file_path, 'rb') as file:
    # Загружаем файл в GridFS
    file_id = fs.put(file, filename='steam_games.json')
    print(f'Large file uploaded with ID: {file_id}')

# Извлечение файла и загрузка данных в коллекцию
file_data = fs.get(file_id)

# Чтение содержимого JSON файла
data = json.load(file_data)

# Вставка данных в коллекцию 'games'
if isinstance(data, list):  # Проверяем, является ли data списком
    db.games.insert_many(data)  # Вставляем все игры
    print(f'Inserted {len(data)} games into the database.')
else:
    print('The JSON file does not contain a list of games.')

# # Удаление файла из GridFS (если это необходимо)
# fs.delete(file_id)
# print('Large file deleted from GridFS.')


Large file uploaded with ID: 6734f854d6ff34dbf56faf1c
The JSON file does not contain a list of games.


In [ ]:
import json

# Путь к вашему файлу steam_games.json
large_file_path = 'steam_games.json'

# Чтение содержимого JSON файла
with open(large_file_path, 'r', encoding='utf-8', errors='replace') as file:
    data = json.load(file)

# Печать типа и структуры данных
print(type(data))
print(data)

In [11]:
import json
import gridfs
from pymongo import MongoClient

# Подключение к MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client.steam  # Убедитесь, что база данных называется steam
collection = db.games  # Коллекция для хранения игр
fs = gridfs.GridFS(db)  # Инициализация GridFS

# Путь к вашему большому файлу
file_path = 'steam_games.json'

# Загружаем файл в GridFS
with open(file_path, 'rb') as file:
    file_id = fs.put(file, filename='steam_games.json')
    print(f'File uploaded to GridFS with ID: {file_id}')

# Чтение файла и вставка данных в коллекцию
with open(file_path, 'r') as file:
    # Предполагаем, что файл содержит массив объектов JSON
    buffer = []
    for line in file:
        try:
            game_data = json.loads(line.strip())  # Загружаем каждую строку как JSON объект
            buffer.append(game_data)

            # Если буфер достигает определенного размера, вставляем данные в коллекцию
            if len(buffer) >= 1000:  # Например, вставляем по 1000 объектов за раз
                result = collection.insert_many(buffer)
                print(f'Inserted {len(result.inserted_ids)} games into the database.')
                buffer.clear()  # Очищаем буфер

        except json.JSONDecodeError as e:
            print(f'Error decoding JSON: {e}')

    # Вставляем оставшиеся данные в буфере
    if buffer:
        result = collection.insert_many(buffer)
        print(f'Inserted {len(result.inserted_ids)} games into the database.')


File uploaded to GridFS with ID: 6735068511c199b79237ae12


UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 5797: character maps to <undefined>

In [7]:
import json
import os

def split_json(input_file, output_dir, max_size):
  """
  Разделяет JSON-файл на несколько меньших файлов.

  Args:
      input_file (str): Путь к исходному JSON-файлу.
      output_dir (str): Каталог для сохранения разделенных файлов.
      max_size (int): Максимальный размер каждого выходного файла в байтах.
  """

  os.makedirs(output_dir, exist_ok=True) # Создать каталог, если его нет

  with open(input_file, 'r', encoding='utf-8', errors='replace') as f:
    data = json.load(f)

  # Разбить данные на части
  parts = []
  current_part = []
  current_size = 0
  for item in data:
    item_size = len(json.dumps(item))
    if current_size + item_size > max_size:
      parts.append(current_part)
      current_part = [item]
      current_size = item_size
    else:
      current_part.append(item)
      current_size += item_size
  parts.append(current_part) # Добавить последнюю часть

  # Сохранить каждую часть в отдельный файл
  for i, part in enumerate(parts):
    with open(os.path.join(output_dir, f'part_{i+1}.json'), 'w', encoding='utf-8') as f:
      json.dump(part, f, indent=2) # Используйте indent для лучшей читаемости

# Пример использования
input_file = 'steam_games-2.json'
output_dir = 'output_files'
max_size = 1024 * 1024 # 1 МБ
split_json(input_file, output_dir, max_size)


In [8]:
import json
import os

def split_json(input_file, output_dir, max_size):
    """
    Разделяет JSON-файл на несколько меньших файлов.

    Args:
        input_file (str): Путь к исходному JSON-файлу.
        output_dir (str): Каталог для сохранения разделенных файлов.
        max_size (int): Максимальный размер каждого выходного файла в байтах.
    """

    os.makedirs(output_dir, exist_ok=True) # Создать каталог, если его нет

    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Разбить данные на части
    parts = []
    current_part = []
    current_size = 0
    for item in data:
        # Переводим JSON-объект в строку и получаем размер в байтах
        item_str = json.dumps(item)
        item_size = len(item_str.encode('utf-8')) # Используем кодировку utf-8

        if current_size + item_size > max_size:
            parts.append(current_part)
            current_part = [item]
            current_size = item_size
        else:
            current_part.append(item)
            current_size += item_size
    parts.append(current_part) # Добавить последнюю часть

    # Сохранить каждую часть в отдельный файл
    for i, part in enumerate(parts):
        with open(os.path.join(output_dir, f'part_{i+1}.json'), 'w', encoding='utf-8') as f:
            json.dump(part, f, indent=2) # Используйте indent для лучшей читаемости

# Пример использования
input_file = 'steam_games-2.json'
output_dir = 'output_files'
max_size = 1024 * 1024 # 1 МБ
split_json(input_file, output_dir, max_size)


In [1]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['users']
persons = db.persons

doc = {
    'autor':'Peter',
    'text':'is cool! Wilberry',
    'date':'2020-01-01',
    'likes': 0,
    'tags':['cool','hot','ice'],
    'comments':['good']
}

persons.insert_one(doc)

for doc in persons.find():
    print(doc)

ImportError: cannot import name 'MongoClient' from partially initialized module 'pymongo' (most likely due to a circular import) (c:\Users\nephelim\Desktop\skill\DataCollection\lesson3\pymongo.py)